In [4]:
pip install tensorflow==2.16.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 837.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 639.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 21.4 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import models , layers
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
print(tf. __version__)

2.16.2


In [3]:
image_width=294
image_height=222
batch_size=32
channels=3
epochs=50

In [4]:
dataset = tf.keras.utils.image_dataset_from_directory(
    "/content/drive/MyDrive/diseases",
    shuffle=True,
    image_size=(image_width,image_height),
    batch_size=32
)

Found 25951 files belonging to 9 classes.


In [5]:
class_names=dataset.class_names
class_names

[' Tinea Ringworm Candidiasis and other Fungal Infections',
 ' Warts Molluscum and other Viral Infections',
 'Basal Cell Carcinoma',
 'Benign Keratosis-like Lesions',
 'Eczema ',
 'Melanocytic Nevi',
 'Melanoma',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Seborrheic Keratoses and other Benign Tumors']

In [6]:
trainingSize=0.8
len(dataset)*trainingSize

648.8000000000001

In [7]:
def get_dataset_parts(ds, train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    ds_size=len(ds)
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)


    train_size=int(ds_size*train_split)
    val_size=int(ds_size*val_split)

    train_ds=ds.take(train_size)

    val_ds= ds.skip(train_size).take(val_size)
    test_ds= ds.skip(train_size).skip(val_size)



    return train_ds , val_ds , test_ds

In [8]:
train_ds,val_ds,test_ds=get_dataset_parts(dataset)

In [9]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
resize_and_rescale=tf.keras.Sequential([
    layers.Resizing(image_width,image_height),
    layers.Rescaling(1.0/255)
])

In [14]:
input_shape=(batch_size,image_width,image_height,channels)
n_classes=len(class_names)
model = tf.keras.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.build(input_shape=input_shape)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (32, 294, 222, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (32, 292, 220, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (32, 146, 110, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (32, 144, 108, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (32, 72, 54, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (32, 70, 52, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (32, 35, 26, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (32, 33, 24, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (32, 16, 12, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (32, 14, 10, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (32, 7, 5, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (32, 5, 3, 64)              │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (32, 2, 1, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (32, 128)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (32, 64)                    │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (32, 9)                     │             585 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 175,945 (687.29 KB)

 Trainable params: 175,945 (687.29 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
history=model.fit(
    train_ds,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
    validation_data=val_ds,
)


Epoch 1/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 573s 608ms/step - accuracy: 0.3728 - loss: 1.7503 - val_accuracy: 0.5185 - val_loss: 1.2176
Epoch 2/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 342s 527ms/step - accuracy: 0.5276 - loss: 1.1773 - val_accuracy: 0.6038 - val_loss: 1.0224
Epoch 3/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 343s 529ms/step - accuracy: 0.5872 - loss: 1.0462 - val_accuracy: 0.6088 - val_loss: 0.9861
Epoch 4/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 340s 525ms/step - accuracy: 0.6145 - loss: 0.9785 - val_accuracy: 0.5949 - val_loss: 0.9847
Epoch 5/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 341s 527ms/step - accuracy: 0.6256 - loss: 0.9384 - val_accuracy: 0.6512 - val_loss: 0.9066
Epoch 6/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 349s 539ms/step - accuracy: 0.6433 - loss: 0.9001 - val_accuracy: 0.6651 - val_loss: 0.8768
Epoch 7/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 343s 529ms/step - accuracy: 0.6604 - loss: 0.8570 - val_accuracy: 0.6813 - val_loss: 0.8321
Epoch 8/50
648/648 ━━━━━━━━━━━━━━━━━━━━ 341s 527ms/step - accuracy: 0.6728 -

In [18]:
scores=model.evaluate(test_ds)


82/82 ━━━━━━━━━━━━━━━━━━━━ 63s 117ms/step - accuracy: 0.8541 - loss: 0.5942


In [ ]:
model_version=1
model.save(f'model{model_version}.h5', save_format='h5')


In [20]:
model.save('/content/drive/MyDrive/myModel.h5', save_format='h5')